# Prédiction de valeurs manquantes : application à des mesures de capteur

Yewan Wang, Guillaume Simon @ Miratlas, Inès Hafassa Maïza @ DataCraft
Juillet 2023

## Thème
Prétraitement des données de séries temporelles pour la complétion des turbulences atmosphériques liées aux conditions météorologiques.

## Contexte
La société "Miratlas" déploie un réseau instrumental mondiale pour surveiller les conditions météorologiques, en particulier les turbulences, afin de caractériser les conditions atmosphériques pour la communication optique en espace libre. Les variables de turbulence sont mesurées en observant les objets célestes brillants dans le ciel, tels que le soleil pendant la journée et Polaris pendant la nuit. Cependant, l'instrument ne peut pas effectuer de mesures efficaces si le soleil ou l'étoile est invisible, par exemple lorsqu'il y a des nuages ou lorsque les objets célestes sont en dessous de l'altitude d'observation. Par conséquent, nous avons de nombreuses valeurs manquantes pour les variables de turbulences. En revanche, la station météorologique obtient normalement des mesures continues sans interruption. Dans cet atelier, nous souhaitons analyser les relations internes et potentielles entre les variables de turbulence et les données météorologiques, afin de déterminer si nous pouvons obtenir des prédictions à court terme des turbulences en utilisant les variables météorologiques pour compléter les measures manquantes de la turbulence.

## Plan
1. Introduction : Présentation de la problématique et des enjeux.
2. Visualization et prétraitement des données : Observation les évolutions et les valeurs manquantes des données.
3. Exploration des données : Analyse statistique des jeux de données, en terme de la corrélation, la saisonalité, les valeurs aberrantes pour explorer les relations potentielles entre les variables météorologiques et les turbulences.
4. Méthodes pre-selectionnées: <br>
    4.1. Modélisation prédictive : Construction d'un modèle prédictif en utilisant des techniques d'apprentissage automatique pour estimer les turbulences à court terme en fonction des données météorologiques.
5. Évaluation des performances : Évaluation du modèle prédictif et discussion des résultats obtenus.
6. Conclusion : Discusion les résultats obtenus ensemble.


## 1. Introduction

### 1.1 Turbulence atmosphérique
La turbulence atmosphérique fait référence à l’instabilité des mouvements de l’air. Comme un faisceau laser se propage dans l’atmosphère, son front d’onde subit des distorsions dues à des inhomogénéités dans l’indice de réfraction de l’air, qui sont causées par des variations de température et de pression. Ces effets sur un système optique peuvent être quantifiés par des paramètres de vision atmosphérique tels que la longueur de cohérence atmosphérique (paramètre Fried, r0) et l’angle isoplanatique.

Source:
- Apprends la science qui sous-tend les quatre principales causes de turbulence: https://parlonssciences.ca/ressources-pedagogiques/les-stim-expliquees/quest-ce-que-la-turbulence-atmospherique#:~:text=La%20turbulence%20en%20air%20clair%20se%20produit%20lorsque%20deux%20masses,de%20l'atmosph%C3%A8re%20appel%C3%A9e%20tropopause.

### 1.2 Description du jeu de données 
1. time: sampling time  datetime64
2. d_ext_temp: external temperature, float64
3. d_humid: relative humidity, float64
4. d_rain_rate：rain rate, float64       
5. d_wind: wind speed, float64       
6. d_wind_dir: wind direction, 0° North, 180° south, float64       
7. __day_r0__: Fried parameter r0 day, float64       
8. __day_see__: Day Seeing, float64       
9. __day_see_stddev__, float64       
10. down_ir: downwelling irrandance, radiation infrared, float64       
11. humid: internal humidity, float64       
12. irrad: irradance, float64       
13. __isoplan__: Night Seeing isoplanetisme angle, Related to scintillation, float64       
14. __night_r0__: Fried parameter r0 night, float64       
15. __night_see__: Night Seeing, float64       
16. press: pression, float64       
17. pyr_temp: Pyrgeomete temperature, at 52° elevation, float64       
18. __scint__: Scintillation night, float64, 0-255       
19. sky_temp: sky temperature, float64       
20. status: four catogeries based on status of DIMM: Day time, Cloudy, Night time, Polaris locked $^*$ , string        
21. transp: Transparency night, float64, 0-255       
22. wat_col_hei: Total Water Column, water vapor, float64    

$^*$ : concerne l'étoile Polaris 

__en gras__ : les variables caractérisant la turbulence




### 1.3 Parameters

In [ ]:
import os

In [ ]:
data_directory = os.getcwd() + os.sep  + "data"  # Directory where we put the data.
path_data_raw = data_directory + os.sep + 'tenerife_2020.csv'  # Path to raw data.
path_data_ext =  data_directory + os.sep + 'tenerife2020_extended.csv'  # Path to extended dataset.

## 2. Visualisation et prétraitement

Observation les évolutions et les valeurs manquantes des données, pretraitement des données brutes.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from utils import *

### 2.1 Importation des données

In [ ]:
cols_names = ['time', 'status', 'd_ext_temp', 'd_humid', 'd_rain_rate', 'd_wind', 'd_wind_dir', 'day_r0', 'day_see', 
            'day_see_stddev', 'down_ir', 'humid', 'irrad', 'isoplan', 'night_r0', 'night_see', 'press',
            'pyr_temp', 'scint', 'sky_temp', 'transp', 'wat_col_hei']
column_types = ['string', 'string', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32',
                'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32']


dtype = dict(zip(cols_names, column_types))
df = pd.read_csv(path_data_raw, usecols=cols_names, dtype=dtype)
df.replace(0, np.nan, inplace=True)
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df['time'] = pd.to_datetime(df['time'], unit='ns')

df[df['time'].dt.month == 1].info()

### 2.2 Valeurs manquantes

In [ ]:
import missingno as msno
missing_df = missingDF(df)
print('Dataframe des variables associées leurs taux de valeurs manquantes')
print(missing_df)
msno.matrix(df[df['time'].dt.month == 7]) # pour juillet


- Les variables continues sans trous (ou presque) sont : __d_ext_temp__ , __d_wind_dir__, __down_ir__, __pyr_temp__, __press__, __sky_temp__, __wat_col_hei__, __humid__, __irrad__

- Le taux de manquants pour les variables à trous qu'on cherche à prédire sont d'env 74%

- Les variables (quasi ou entièrement) vides sont __isoplan__ et __d_rain_rate__

On va supprimer ces deux dernières variables car elles ne nous serviront pas.


In [ ]:
df = df.drop(['isoplan','d_rain_rate'],axis=1)

### 2.3 Ajouter les variables supplémentaires

In [ ]:
if os.path.exists(path_data_ext):
    df = pd.read_csv(path_data_ext)
    df.time = pd.to_datetime(df.time)
else:
    df = add_features_from_raw_data(df, dic_location['tenerife']) # Ajouter: hour_of_day, month, season, sun_alt
    df.to_csv(path_data_ext)
print(df[df['time'].dt.month == 1].info())

### 2.3 On crée des dataframes par catégorie
Cela pourra être utile pour l'observation des données, pour la création de modèle etc.

__Decomposition par saison__

In [ ]:
#hiver = pd.concat([df[(df['time'].dt.month >= 12) & (df['time'].dt.day >=1)], df[(df['time'].dt.month <= 2) & (df['time'].dt.day <= 28)]])
#printemps = df[(df['time'].dt.month >= 3) & (df['time'].dt.day >=1) & (df['time'].dt.month <= 5) & (df['time'].dt.day <= 31)]
#ete = df[(df['time'].dt.month >= 6) & (df['time'].dt.day >=1) & (df['time'].dt.month <= 8) & (df['time'].dt.day <= 31)]
#automne = df[(df['time'].dt.month >= 9) & (df['time'].dt.day >=1) & (df['time'].dt.month <= 11) & (df['time'].dt.day <= 31)]
df = pd.read_csv('data/tenerife2020_extended.csv', dtype=dtype)
df.drop(columns=['Unnamed: 0'], inplace=True)
df['time'] = pd.to_datetime(df['time'], unit='ns')
print(df.info())
hiver = df[df['season'] == 1] #12, 1, 2
printemps = df[df['season'] == 2] #3, 4, 5
ete = df[df['season'] == 3] #6, 7, 8
automne = df[df['season'] == 4] #9, 10, 11 

In [ ]:
#hiver,automne,printemps,été
#missingDF(printemps)
msno.matrix(printemps)
msno.matrix(ete)
msno.matrix(automne)
msno.matrix(hiver)

__Printemps__ :  peu de valeurs manquantes (__day_r0__, __night_r0__) et peu de val manquantes chez les variables servant à prédire 

__Ete__ : peu de val manquante chez __day_r0__ mais bcp chez __night_r0__ et chez les variables pour prédire

__Automne__ : pas mal de valeurs manquantes (__day_r0__, __night_r0__) mais peu de val manquantes chez les variables servant à prédire

__Hiver__ : bcp de valeurs manquantes autant chez les variables à prédire que chez les variables pouvant servir à prédire (__scint__, __transp__, __d_wind__ etc) = bof

__Decomposition par capteurs__

In [ ]:
features_all = ['day_r0', 'day_see', 'day_see_stddev', 'night_r0', 'night_see', 'scint', 'transp', 'd_wind_dir', 'humid', 'd_humid', 'press', 'irrad', 'sky_temp', 'wat_col_hei', 'pyr_temp', 'down_ir']
features_turbulence_day = ['day_r0', 'day_see', 'day_see_stddev']  # Scintillation solar sensor
features_turbulence_night = ['night_r0', 'night_see', 'scint', 'transp'] # DIMM
features_weather = ['d_wind_dir', 'humid', 'd_humid', 'press', 'irrad'] # Weather station
features_thermal = ['sky_temp', 'wat_col_hei', 'pyr_temp', 'down_ir']  # 1,2: zenith sensor, 10°FoV; 3,4: 52° sensor, 40°FoV

In [ ]:
corr_table = correlation_table(df, 0.7, features_all)
print_results_table(corr_table, ['Param1', 'Param2', 'Spearman'])

### 2.4 Fonction de plotting

In [ ]:
# Dessinez les courbes des variables de juillet.
df_month = df[(df['time'].dt.month == 9)]
plot_one_param(df_month, 'day_r0')
plot_one_param(df_month, 'night_r0')

In [ ]:
plot_two_params(df, 'sun_alt', 'sky_temp')

In [ ]:
plot_one_params_based_categories(automne, 'sky_temp', 'status')
plot_one_params_based_categories(df, 'humid', 'season')

## 3. Statistiques descriptives

#### Variables quantitatives

In [ ]:
df.describe()

__Affichage des box-plot__ : 

In [ ]:
df[['day_r0','night_r0']].boxplot()

__Affichage des histogrammes__ :

In [ ]:
sns.histplot(df.night_r0) #day_r0
plt.show()
sns.histplot(printemps.night_r0) #night_r0
plt.show()
sns.histplot(ete.night_r0) #night_r0
plt.show()
sns.histplot(automne.night_r0) #night_r0
plt.show()
sns.histplot(hiver.night_r0) #night_r0
plt.show()

#### Variables qualitatives

In [ ]:
print("L'hiver")
t = pd.crosstab(hiver.status, "freq", normalize = True)
t.plot.pie(subplots=True, figsize = (4, 4))
plt.show()

print("L'automne")
t = pd.crosstab(automne.status, "freq", normalize = True)
t.plot.pie(subplots=True, figsize = (4, 4))
plt.show()

print("Le printemps")
t = pd.crosstab(printemps.status, "freq", normalize = True)
t.plot.pie(subplots=True, figsize = (4, 4))
plt.show()

print("L'été")
t = pd.crosstab(ete.status, "freq", normalize = True)
t.plot.pie(subplots=True, figsize = (4, 4))
plt.show()

In [ ]:
#df,hiver,automne,printemps,ete

hiver.status.unique()
print("L'hiver \n", pd.crosstab(hiver.status, "freq"),"\n")

automne.status.unique()
print("L'automne \n", pd.crosstab(automne.status, "freq"),"\n")

printemps.status.unique()
print("Le printemps \n", pd.crosstab(printemps.status, "freq"),"\n")

ete.status.unique()
print("L'été \n", pd.crosstab(ete.status, "freq"))

#### Description conjointe de caractères quantitatifs

In [ ]:
print("hiver")
plot_corr(hiver)
print("automne")
plot_corr(automne)
print("printemps")
plot_corr(printemps)
print("ete")
plot_corr(ete)
print("sur l'année")
plot_corr(df)

## Résumé des corrélations ( $\geq 0.84 \% $ ) : 

Variables corrélées pour toute saison : `sky_temp ~ wat_col_hei` et `pyt_temp ~ down_ir`

### Hiver : 

`transp ~ scint`

### Automne : 

`transp ~ scint`, `d_ext_temp ~ sky_temp`,`d_ext_temp ~ wat_col_hei`

### Printemps : 

### Eté : 

`transp ~ scint`, `d_ext_temp ~ sky_temp`,`d_ext_temp ~ wat_col_hei`, 


In [ ]:
Hiver,Automne,Printemps,Ete = [[hiver,'hiver'],[automne,'automne'],[printemps,'printemps'],[ete,'été']]

plot_corr_saison_variable('wat_col_hei', 'sky_temp', Printemps)
plot_corr_saison_variable('down_ir', 'pyr_temp', Printemps)

### Description conjointe d’un caractère quantitatif et d’un caractère qualitatif

In [ ]:
df.groupby("status").mean()

In [ ]:
#ex ou on se concentre sur une variable avec sky_temp

df.groupby("status")["sky_temp"].agg([np.mean, np.std, np.median, np.min, np.max])

In [ ]:
sns.histplot(data = df, x="sky_temp", hue = "status", multiple = "stack")

#### Description conjointe de deux caractères quantitatifs et d’un caractère qualitatif

In [ ]:
sns.relplot(x = "down_ir", y = "pyr_temp", hue = "status", data = df, height = 6, s = 30)

In [ ]:
sns.lmplot(data = df, x="sky_temp", y = "wat_col_hei", hue = "status", col = "status")

## 3. Tendance et saisonnalité

### Affichage de la tendance

In [ ]:
df['day'] = df['time'].dt.day
df['month'] = df['time'].dt.month
month = df['month'].unique()

plot_df(df, x=df.time, y=df.sky_temp, ylabel = "Sky temperature" ,title='Sky temperature plot',color = 'green')  

plot_df(df, x=df.time, y=df.pyr_temp, ylabel = "Pyr temperature" ,title='Pyr temperature plot', color = 'blue')  

plot_df(df, x=df.time, y=df.day_r0, ylabel = "Day r0" ,title='Day r0 plot', color = 'purple')  

plot_df(df, x=df.time, y=df.night_r0, ylabel = "Night r0" ,title='Night r0 plot')  

plot


### Affichage de la saisonnalité (ex : en juillet)

In [ ]:
plt.plot('day_r0', data=df.loc[~df.month.isin([7]), :],)
plt.title('Month-wise plot day r0 \n(The Seasonality)', fontsize=12)
plt.show()
plt.plot('night_r0', data=df.loc[~df.month.isin([7]), :],)
plt.title('Month-wise plot night r0 \n(The Seasonality)', fontsize=12)
plt.show()

# Mise en oeuvre algorithme de prédiction des valeurs manquantes

mettre en œuvre des algorithmes de traitement de données pour compléter les valeurs manquantes de la turbulence en utilisant les variables météorologiques, thermiques et supplémentaires. 
Divisez le jeu de données en deux en fonction de la période de la journée (jour et nuit) en utilisant la variable "sun_alt". Les données pour la journée sont "sun_alt > 0°" et celles pour la nuit sont "sun_alt < 0°"

Les variables à compléter: 
- Turbulence de la journée: day_r0 (Quand le soleil est au-dessus de l’horizon -> [sun_alt > 0°])
- Turbulence de la nuit: night_r0 (Quand le soleil est au-dessous de l’horizon -> [sun_alt < 0°])

In [ ]:
#Les Entrées du modèle: 
features_weather = ['d_wind_dir', 'humid', 'd_humid', 'press', 'irrad'] 
features_thermal = ['sky_temp', 'wat_col_hei', 'pyr_temp', 'down_ir']
features_extended = ['sun_alt', 'month', 'hour_of_day', 'season']

## Sujet 1: Imputation les données manquantes pour le variable "day_r0"

In [ ]:
# Selection des données de la journée
vars_day = ['day_r0']
vars_day.extend(features_weather)
vars_day.extend(features_thermal)
vars_day.extend(features_extended)
print(vars_day)
df_jour = df.loc[df['sun_alt'] >= 0, vars_day]
print(df_jour.info())
df_jour['day_r0'].plot(figsize=(15, 6))

## Sujet 2: Imputation les données manquantes pour le variable "night_r0"

In [ ]:
# Selection des données de la nuit
vars_night = ['night_r0']
vars_night.extend(features_weather)
vars_night.extend(features_thermal)
vars_night.extend(features_extended)

df_night = df.loc[df['sun_alt'] < 0, vars_night]
print(df_night.info())
df_night['night_r0'].plot(figsize=(15, 6))

In [ ]:
list_cols = list(df.columns)
list_index = list(df.index)

In [ ]:
dfComplet = complete_dataset(df,5)
dfMissing = missing_dataset(df,5)

In [ ]:
missingDF(dfComplet)

On a un dataset complet qui nous servira pour entrainer notre modèle

In [ ]:
split = int(0.80*len(dfComplet))
split 